# Percobaan Data Sampel Holt-Winters

Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import*

Load Dataset

In [2]:
#importing data
df = pd.read_csv('data.csv')
data = df.iloc[:,1]
data

0      12172
1      14270
2      15600
3      14998
4      16648
       ...  
103    24623
104    18913
105    21518
106    20798
107    22431
Name: DataAktual, Length: 108, dtype: int64

Inialisasi Nilai Awal, Smoothing & Prediksi

In [3]:
# 1. Menghitung Nilai Awal Pemodelan
# 1a. Menghitung Nilai Smoothing
def smoothingAwal(data):
    data_asli = data.iloc[:12,]
    S = []
    total = 0
    for x in data_asli:
        total = total + x
    smoothing = 1/12*(total)
    S.append(smoothing)
    return S

#Sl = 1/l(X1+X2+...+Xl)

# 1b. Menghitung Nilai Awal Musiman
def nilaiAwalMusimanMultiplicative(data, nilaiSmoothing):
    L = []
    data_asli = data.iloc[:12,]
    for x in data_asli:
        nilaiMusim = x/nilaiSmoothing[0]
        L.append(nilaiMusim)
    return L

#Lt = Xt/Sl

# 1b. Menghitung Nilai Awal Musiman
def nilaiAwalMusimanAdditive(data, nilaiSmoothing):
    L = []
    data_asli = data.iloc[:12,]
    for x in data_asli:
        nilaiMusim = x-nilaiSmoothing[0]
        L.append(nilaiMusim)
    return L

#Lt = Xt-Sl

# 1c. Menghitung Nilai Awal Tren
def nilaiTrenAwal(data):
    tren = []
    totalTren = 0
    data_asli = data.iloc[:12,]
    data_tren = data.iloc[12:24,]
    for x in range(12):
        total = (data_tren[12+x]-data_asli[x])/12
        totalTren = totalTren + total
    trenAwal = 1/12 * (totalTren)
    tren.append(trenAwal)
    return tren

#Bl= 1/12 (Xl+1-X1)/l + (Xl+2-X2)/l+...+(Xl+l-Xl)/l

# 2. Menghitung Nilai Smoothing Additive
# 2a. menghitung nilai Smoothing Keseluruhan
def smoothingKeseluruhanAdditive(alpa, data ,nilaiSmoothing, nilaiMusiman, nilaiTren, x):
    nilai_smooth =((alpa * (data[x+12] - nilaiMusiman[x])) + ((1-alpa) * (nilaiSmoothing[x] + nilaiTren[x])))
    return  nilai_smooth

#St = a(Xt-lt-l)+(1-a)(St-1+Bt-1)

# 2b. menghitung nilai tren
def trendSmoothingAdditive(beta, nilaiSmoothing, nilaiTren,x):
    nilai_tren_smooth = ((beta * (nilaiSmoothing[x+1]-nilaiSmoothing[x])) + ((1-beta)*nilaiTren[x]))
    return nilai_tren_smooth

#Bt = b(St-St-1)+(1-b)Bt-1

# 2c. menghitung nilai Musiman
def nilaiMusimanSmoothingAdditive(gamma, data, nilaiSmoothing, nilaiMusiman, x):
    nilai_musim = ((gamma * (data[x+12]-nilaiSmoothing[x+1])) + ((1-gamma)*(nilaiMusiman[x])))
    return nilai_musim

#lt = y(Xt-St)+(1-y)lt-L

# 3. Menghitung Nilai Smoothing Multipicative
# 3a. menghitung nilai Smoothing Keseluruhan
def smoothingKeseluruhanMultiplicative(alpa, data, nilaiSmoothing, nilaiMusiman, nilaiTren, x):
    nilai_smooth = ((alpa * (data[x+12] / nilaiMusiman[x])) + ((1-alpa) * (nilaiSmoothing[x] + nilaiTren[x])))
    return nilai_smooth

#St = a(Xt/lt-l)+(1-a)(St-1+Bt-1)

# 3b. menghitung nilai tren
def trendSmoothingMultiplicative(beta, nilaiSmoothing, nilaiTren, x):
    nilai_tren_smooth = ((beta * (nilaiSmoothing[x+1]-nilaiSmoothing[x])) + ((1-beta)*nilaiTren[x]))
    return nilai_tren_smooth

#Bt = b(St-St-1)+(1-b)Bt-1

# 3c. menghitung nilai Musiman
def nilaiMusimanSmoothingMultiplicative(gamma, data, nilaiSmoothing, nilaiMusiman, x):
    nilai_musim = ((gamma * (data[x+12] / nilaiSmoothing[x+1])) + ((1-gamma)*(nilaiMusiman[x])))
    return nilai_musim

#lt = y(Xt/St)+(1-y)lt-L

# 4. Menghitung Prediksi
# menghitung nilai Prediksi Additive
def nilaiPredictAdditive(nilaiSmoothing, nilaiTren, nilaiMusiman, x):
    nilai_predict = nilaiSmoothing[x] + nilaiTren[x] + nilaiMusiman[x]
    return nilai_predict

#Ft+m = St+Btm+lt-l+m

# menghitung nilai Prediksi Multipicative
def nilaiPredictMultiplicative(nilaiSmoothing, nilaiTren, nilaiMusiman, x):
    nilai_predict = ((nilaiSmoothing[x] + nilaiTren[x]) * (nilaiMusiman[x]))
    return nilai_predict

#Ft+m = (St+Btm)lt-l+m


# Nilai Smoothing berdasarkan Iterasi dengan nilai parameter yang ditentukan

Additive

Inialisasi Smoothing, Trend, dan Musiman untuk data 2012-2019, dimana nilai parameter yang disarankan : alpha < 0.5, beta < 0.5, gamma > 0.5 dan iterasi = 1

In [4]:
hasilPrediksiTrainingAdd=[]
smoothingAdd = []
musimanAdd = []
trenAdd = []
nilaiAlpa = []
nilaiBeta = []
nilaiGamma = []
nilaiMAPEAdditive = []

      
# import decimal
alpa = 0.241
nilaiAlpa.append(alpa)
beta = 0.001
nilaiBeta.append(beta)
gamma = 0.724
nilaiGamma.append(gamma)
    
nilaiSmoothingAdditive = smoothingAwal(data)
print("Nilai Smoothing Awal Additive = ", nilaiSmoothingAdditive)
    
nilaiMusimanAdditive = nilaiAwalMusimanAdditive(data, nilaiSmoothingAdditive)
print("Nilai Musiman Awal Additive = ", nilaiMusimanAdditive)
    
nilaiTrenAdditive = nilaiTrenAwal(data)
print("Nilai Tren Awal Additive = ", nilaiTrenAdditive)
    
data_pakai = data.iloc[12:,]
predictAdditive=[]
    
for x in range(0, len(data_pakai)):
    #Additive
    nilai_smooth_additive = smoothingKeseluruhanAdditive(alpa, data_pakai ,nilaiSmoothingAdditive, nilaiMusimanAdditive, nilaiTrenAdditive, x)
    nilaiSmoothingAdditive.append(nilai_smooth_additive)

    nilai_tren_additive = trendSmoothingAdditive(beta, nilaiSmoothingAdditive, nilaiTrenAdditive, x)
    nilaiTrenAdditive.append(nilai_tren_additive)

    nilai_musim_additive = nilaiMusimanSmoothingAdditive(gamma, data_pakai, nilaiSmoothingAdditive, nilaiMusimanAdditive, x)
    nilaiMusimanAdditive.append(nilai_musim_additive)

    nilai_predict_additive = nilaiPredictAdditive(nilaiSmoothingAdditive, nilaiTrenAdditive, nilaiMusimanAdditive, x)
    predictAdditive.append(nilai_predict_additive)

MAPEAdditive = 0
errorAdditive = 0
    
for i in range (0, len(data_pakai)):
    totalAdditive = abs((predictAdditive[i] - data_pakai[i+12])/predictAdditive[i]) 
    errorAdditive = errorAdditive + totalAdditive
        
MAPEAdditive = errorAdditive / (len(data_pakai))
nilaiMAPEAdditive.append(MAPEAdditive)
    
print("Nilai alpa >>>>>>>", alpa)
print("Nilai beta >>>>>>>", beta)
print("Nilai gamma >>>>>>>", gamma)
print("MAPE Additive >>>>>>>", MAPEAdditive*100, "\n")
    
smoothingAdd.append(nilaiSmoothingAdditive)
musimanAdd.append(nilaiMusimanAdditive)
trenAdd.append(nilaiTrenAdditive)
hasilPrediksiTrainingAdd.append(predictAdditive)

Nilai Smoothing Awal Additive =  [16054.166666666666]
Nilai Musiman Awal Additive =  [-3882.166666666666, -1784.166666666666, -454.16666666666606, -1056.166666666666, 593.8333333333339, -75.16666666666606, 1591.833333333334, -2829.166666666666, -1537.166666666666, -662.1666666666661, 3852.833333333334, 6241.833333333334]
Nilai Tren Awal Additive =  [91.63194444444443]
Nilai alpa >>>>>>> 0.241
Nilai beta >>>>>>> 0.001
Nilai gamma >>>>>>> 0.724
MAPE Additive >>>>>>> 10.479918119683079 



In [5]:
smoothingadd = nilaiSmoothingAdditive[0]
df = pd.DataFrame(smoothingadd,
            columns=["Nilai Smoothing"],
            index=["Januari"])
df

,Nilai Smoothing
Januari,16054.166667


In [6]:
musimanadd = nilaiMusimanAdditive[0:12]
df = pd.DataFrame(musimanadd,
            columns=["Nilai Musiman"],
            index=['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni', 'Juli', 
                        'Agustus', 'September', 'Oktober', 'November', 'Desember'])
df

,Nilai Musiman
Januari,-3882.166667
Februari,-1784.166667
Maret,-454.166667
April,-1056.166667
Mei,593.833333
Juni,-75.166667
Juli,1591.833333
Agustus,-2829.166667
September,-1537.166667
Oktober,-662.166667


In [7]:
trendadd = nilaiTrenAdditive[0]
df = pd.DataFrame(trendadd,
            columns=["Nilai Smoothing"],
            index=["Januari"])
df

,Nilai Smoothing
Januari,91.631944


## Multiplicative

Inialisasi Smoothing, Trend, dan Musiman untuk data 2012-2019, dimana nilai parameter yang disarankan : alpha < 0.5, beta < 0.5, gamma > 0.5 dan iterasi = 1

In [8]:
hasilPrediksiTrainingMul=[]
smoothingMul = []
musimanMul = []
trenMul = []
nilaiAlpa = []
nilaiBeta = []
nilaiGamma = []
nilaiMAPEMultiplicative = []

   
# import decimal
alpa = 0.241
nilaiAlpa.append(alpa)
beta = 0.001
nilaiBeta.append(beta)
gamma = 0.724
nilaiGamma.append(gamma)
    
nilaiSmoothingMultiplicative = smoothingAwal(data)
print("Nilai Smoothing Awal Multiplicative = ", nilaiSmoothingMultiplicative)
    
nilaiMusimanMultiplicative = nilaiAwalMusimanMultiplicative(data, nilaiSmoothingMultiplicative)
print("Nilai Musiman Awal Multiplicative = ", nilaiMusimanMultiplicative)
    
nilaiTrenMultiplicative = nilaiTrenAwal(data)
print("Nilai Tren Awal Multiplicative = ", nilaiTrenMultiplicative)

data_pakai = data.iloc[12:,]
predictMultiplicative=[]
    
for x in range(0, len(data_pakai)):
    #Multiplicative
    nilai_smooth_multiplicative = smoothingKeseluruhanMultiplicative(alpa, data_pakai ,nilaiSmoothingMultiplicative, nilaiMusimanMultiplicative, nilaiTrenMultiplicative, x)
    nilaiSmoothingMultiplicative.append(nilai_smooth_multiplicative)

    nilai_tren_multiplicative = trendSmoothingMultiplicative(beta, nilaiSmoothingMultiplicative, nilaiTrenMultiplicative, x)
    nilaiTrenMultiplicative.append(nilai_tren_multiplicative)

    nilai_musim_multiplicative = nilaiMusimanSmoothingMultiplicative(gamma, data_pakai, nilaiSmoothingMultiplicative, nilaiMusimanMultiplicative, x)
    nilaiMusimanMultiplicative.append(nilai_musim_multiplicative)

    nilai_predict_multiplicative = nilaiPredictMultiplicative(nilaiSmoothingMultiplicative, nilaiTrenMultiplicative, nilaiMusimanMultiplicative, x)
    predictMultiplicative.append(nilai_predict_multiplicative)
      
MAPEMultiplicative = 0
errorMultiplicative = 0
    
for i in range (0, len(data_pakai)):
    totalMultiplicative = abs((predictMultiplicative[i] - data_pakai[i+12])/predictMultiplicative[i])
    errorMultiplicative = errorMultiplicative + totalMultiplicative
    
MAPEMultiplicative = errorMultiplicative / (len(data_pakai))
nilaiMAPEMultiplicative.append(MAPEMultiplicative)
print("Nilai alpa >>>>>>>", alpa)
print("Nilai beta >>>>>>>", beta)
print("Nilai gamma >>>>>>>", gamma)
print("MAPE Multiplicative >>>>>>>", MAPEMultiplicative*100, "\n")
       
smoothingMul.append(nilaiSmoothingMultiplicative)
musimanMul.append(nilaiMusimanMultiplicative)
trenMul.append(nilaiTrenMultiplicative)
hasilPrediksiTrainingMul.append(predictMultiplicative)

Nilai Smoothing Awal Multiplicative =  [16054.166666666666]
Nilai Musiman Awal Multiplicative =  [0.7581832338437582, 0.8888658188424604, 0.9717103555670906, 0.934212302102258, 1.036989358941085, 0.9953179340773424, 1.099153906047236, 0.8237736828445368, 0.904251232805606, 0.9587542174928627, 1.2399896184791073, 1.3887983389566572]
Nilai Tren Awal Multiplicative =  [91.63194444444443]
Nilai alpa >>>>>>> 0.241
Nilai beta >>>>>>> 0.001
Nilai gamma >>>>>>> 0.724
MAPE Multiplicative >>>>>>> 10.431314545700925 



In [9]:
smoothingmul = nilaiSmoothingMultiplicative[0]
df = pd.DataFrame(smoothingmul,
            columns=["Nilai Smoothing"],
            index=["Januari"])
df

,Nilai Smoothing
Januari,16054.166667


In [10]:
musimanmul = nilaiMusimanMultiplicative[0:12]
df = pd.DataFrame(musimanmul,
            columns=["Nilai Musiman"],
            index=['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni', 'Juli', 
                        'Agustus', 'September', 'Oktober', 'November', 'Desember'])
df

,Nilai Musiman
Januari,0.758183
Februari,0.888866
Maret,0.971710
April,0.934212
Mei,1.036989
Juni,0.995318
Juli,1.099154
Agustus,0.823774
September,0.904251
Oktober,0.958754


In [11]:
trendmul = nilaiTrenMultiplicative[0]
df = pd.DataFrame(trendmul,
            columns=["Nilai Smoothing"],
            index=["Januari"])
df

,Nilai Smoothing
Januari,91.631944


Hasil Prediksi Additive untuk tahun 2020

In [12]:
MAPEterkecilAdd = nilaiMAPEAdditive[0]
alpaFinalAdd = nilaiAlpa[0]
betaFinalAdd = nilaiBeta[0]
gammaFinalAdd = nilaiGamma[0]
smoothFinalAdd = smoothingAdd[0]
musimanFinalAdd = musimanAdd[0]
trenFinalAdd = trenAdd[0]

smoothAddFinal = [smoothFinalAdd[len(data_pakai)]]
musimanAddFinal = musimanFinalAdd[(len(data_pakai)):]
trenAddFinal = [trenFinalAdd[len(data_pakai)]]
prediksiAdd = []
for x in range(12):
    pred = smoothAddFinal[x] + musimanAddFinal[x] + trenAddFinal[x]
    prediksiAdd.append(pred)
    smoothing = ((alpaFinalAdd*(prediksiAdd[x]-musimanAddFinal[x]))+ ((1-alpaFinalAdd)*(smoothAddFinal[x]-trenAddFinal[x])))
    smoothAddFinal.append(smoothing)
    trensmoothing = ((betaFinalAdd*((smoothAddFinal[x+1])-(smoothAddFinal[x]))) + ((1-betaFinalAdd)*trenAddFinal[x]))
    trenAddFinal.append(trensmoothing) 
    musim = (gammaFinalAdd*(prediksiAdd[x]-smoothAddFinal[x+1])) + ((1-gammaFinalAdd)*musimanAddFinal[x])
    musimanAddFinal.append(musim)

In [13]:
predikadd = prediksiAdd
df = pd.DataFrame(predikadd, 
                  columns=["Prediksi Wisatawan"],
                  index=['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni', 'Juli', 
                        'Agustus', 'September', 'Oktober', 'November', 'Desember'])
df

,Prediksi Wisatawan
Januari,17905.258379
Februari,21238.167489
Maret,21692.803852
April,20609.011214
Mei,15963.043834
Juni,18907.559864
Juli,21372.701639
Agustus,24078.838206
September,18915.136483
Oktober,19633.048256


Hasil Prediksi Multiplicative untuk tahun 2020

In [14]:
MAPEterkecilMul = nilaiMAPEMultiplicative[0]
alpaFinalMul = nilaiAlpa[0]
betaFinalMul = nilaiBeta[0]
gammaFinalMul = nilaiGamma[0]
smoothFinalMul = smoothingMul[0]
musimanFinalMul = musimanMul[0]
trenFinalMul = trenMul[0]

smoothMulFinal = [smoothFinalMul[len(data_pakai)]]
musimanMulFinal = musimanFinalMul[(len(data_pakai)):]
trenMulFinal = [trenFinalMul[len(data_pakai)]]
prediksiMul = []
for x in range(12):
    predik = (smoothMulFinal[x] + trenMulFinal[x])*musimanMulFinal[x]
    prediksiMul.append(predik)
    smoothing = ((alpaFinalMul * (prediksiMul[x]/musimanMulFinal[x])) + ((1-alpaFinalMul) * (smoothMulFinal[x] + trenMulFinal[x])))
    smoothMulFinal.append(smoothing)
    trensmoothing = ((betaFinalMul*((smoothMulFinal[x+1])-(smoothMulFinal[x]))) + ((1-betaFinalMul)*trenMulFinal[x]))
    trenMulFinal.append(trensmoothing)
    musim = (gammaFinalMul*(prediksiMul[x]/smoothMulFinal[x+1])) + ((1-gammaFinalMul)*musimanMulFinal[x])
    musimanMulFinal.append(musim)

In [15]:
data = prediksiMul
df = pd.DataFrame(data,
                  columns=["Prediksi Wisatawan"],
                  index=['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni', 'Juli', 
                        'Agustus', 'September', 'Oktober', 'November', 'Desember'])
df

,Prediksi Wisatawan
Januari,17581.591558
Februari,21219.214304
Maret,21922.849297
April,20872.345913
Mei,16294.859694
Juni,19468.101935
Juli,22244.573700
Agustus,25192.506246
September,20020.644471
Oktober,20733.780115
